In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#format for floats
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
df0 = pd.read_csv('../Neueda2023/Datasets/2010.csv')
df1 = pd.read_csv('../Neueda2023/Datasets/2011.csv')
df2 = pd.read_csv('../Neueda2023/Datasets/2012.csv')
df3 = pd.read_csv('../Neueda2023/Datasets/2013.csv')
df4 = pd.read_csv('../Neueda2023/Datasets/2014.csv')
df5 = pd.read_csv('../Neueda2023/Datasets/2015.csv')
df6 = pd.read_csv('../Neueda2023/Datasets/2016.csv')

In [4]:
df_combined = pd.concat([df0, df1, df2, df3, df4, df5, df6])

In [5]:
df_combined.groupby(['Year', 'State']).mean()

Days with AQI  Good Days  Moderate Days  \
Year State                                                    
2010 Alabama               282.26     164.89         113.05   
     Alaska                262.12     222.00          35.00   
     Arizona               309.54     199.08         102.23   
     Arkansas              208.24     138.53          67.35   
     California            350.40     221.94         102.34   
...                           ...        ...            ...   
2016 Virginia              254.85     227.79          25.97   
     Washington            348.76     317.45          30.34   
     West Virginia         272.57     237.79          34.07   
     Wisconsin             282.93     243.26          35.85   
     Wyoming               329.39     286.22          42.94   

                    Unhealthy for Sensitive Groups Days  Unhealthy Days  \
Year State                                                                
2010 Alabama                                       4.00            0.32   
     Alaska                                        4.12            1.00   
     Arizona                                       7.92            0.15   
     Arkansas                                      2.24            0.12   
     California                                   19.89            5.32   
...                                                 ...             ...   
2016 Virginia                                      1.09            0.00   
     Washington                                    0.79            0.17   
     West Virginia                                 0.71            0.00   
     Wisconsin                                     3.56            0.26   
     Wyoming                                       0.17            0.06   

                    Very Unhealthy Days  Hazardous Days  Max AQI  \
Year State                                                         
2010 Alabama                       0.00            0.00   112.26   
     Alaska                        0.00            0.00    88.50   
     Arizona                       0.00            0.15   233.46   
     Arkansas                      0.00            0.00   100.00   
     California                    0.47            0.43   305.60   
...                                 ...             ...      ...   
2016 Virginia                      0.00            0.00    91.26   
     Washington                    0.00            0.00    88.17   
     West Virginia                 0.00            0.00    95.86   
     Wisconsin                     0.00            0.00   118.59   
     Wyoming                       0.00            0.00    94.78   

                    90th Percentile AQI  Median AQI  Days CO  Days NO2  \
Year State                                                               
2010 Alabama                      70.32       46.16     0.05      0.00   
     Alaska                       44.88       19.75     2.38      0.00   
     Arizona                      70.38       42.92     0.00      5.62   
     Arkansas                     63.94       43.71     0.00      2.94   
     California                   80.60       45.30     0.15      8.91   
...                                 ...         ...      ...       ...   
2016 Virginia                     45.88       30.03     1.21     31.09   
     Washington                   44.24       24.10     0.03      4.03   
     West Virginia                52.57       35.43     4.93      0.00   
     Wisconsin                    57.56       35.96     0.00      1.33   
     Wyoming                      51.06       37.28     0.00      1.78   

                    Days Ozone  Days PM2.5  Days PM10  
Year State                                             
2010 Alabama            131.05      142.26       8.89  
     Alaska              59.38      191.00       9.38  
     Arizona            191.23       22.77      89.92  
     Arkansas           110.35       94.94       0.00  
     California         222.55      105.

In [8]:
!pip install pyxlsb

In [6]:
df_ev = pd.read_excel("./Datasets/vehicle.xlsb", sheet_name="County")
df_ev

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Percentage of vehicles in each jurisdiction,Unnamed: 8,Unnamed: 9,...,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50
0,state_abbr,geoid,county_id,county_name,fuel_type_org,fuel_type,class,before 1980,1980-99,1990-99,...,"2,009.00","2,010.00","2,011.00","2,012.00","2,013.00","2,014.00","2,015.00","2,016.00","2,017.00","2,018.00"
1,AL,1001,1001,Autauga,BI,Other/Unknown,Car,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AL,1001,1001,Autauga,BI,Other/Unknown,Truck,NaN,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AL,1001,1001,Autauga,DIES,Diesel vehicle,Car,NaN,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN
4,AL,1001,1001,Autauga,DIES,Diesel vehicle,Truck,NaN,0.00,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50811,WY,56045,56045,Weston,GAS,Gasoline vehicle,Unknown,0.05,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50812,WY,56045,56045,Weston,HYBRID ELECTRIC VEHICLE,Hybrid electric vehicle,Car,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN
50813,WY,56045,56045,Weston,HYBRID ELECTRIC VEHICLE,Hybrid electric vehicle,Truck,NaN,NaN,NaN,...,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50814,WY,56045,56045,Weston,PLUG IN HYBRID ELECTRIC VEHICLE,Plug-in hybrid electric vehicle,Car,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.00,NaN,NaN,0.00,NaN,NaN


In [7]:
state_codes = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}

In [8]:
df_combined["State"] = df_combined["State"].map(state_codes)

In [9]:
df_combined

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10
0,AL,Baldwin,2010,274,188,80,6,0,0,0,150,71,43,0,0,192,82,0
1,AL,Clay,2010,113,79,34,0,0,0,0,86,60,42,0,0,0,113,0
2,AL,Colbert,2010,355,208,146,1,0,0,0,105,67,47,0,0,114,241,0
3,AL,DeKalb,2010,358,248,106,4,0,0,0,101,74,43,0,0,303,55,0
4,AL,Elmore,2010,241,181,55,5,0,0,0,112,71,43,0,0,241,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025,WY,Sublette,2016,366,297,68,1,0,0,0,102,61,44,0,0,351,15,0
1026,WY,Sweetwater,2016,366,304,62,0,0,0,0,100,58,44,0,0,327,4,35
1027,WY,Teton,2016,366,331,35,0,0,0,0,99,50,41,0,0,351,15,0
1028,WY,Uinta,2016,366,341,25,0,0,0,0,87,49,42,0,0,359,0,7
